In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[ ]:

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [1]:
# change values in 'CONTROL' column to show more simply what is going on
def control_name(file, keep_list):
    control_dict = {1:'Public', 2:'Private, NFP',3:'Private, FP', -3:'Unknown'}
    new_hd = file.replace({keep_list[keep_list.index('CONTROL')]:control_dict})
    return new_hd

In [ ]:
# clean the 'hd' table containing information about each university, getting only the columns that are relevant.
def hdcleaner(year):
    import pandas as pd
    hd = pd.read_csv('/home/u1033387/FinalProject/histdata/hd{}.csv'.format(year),encoding='ISO-8859-1', index_col=False)
    for x in range(len(hd.columns.values)):
        hd.columns.values[x] = hd.columns.values[x].upper()
    keep_list = ['UNITID','INSTNM', 'STABBR','CONTROL']
    
    new_hd = hd[keep_list].copy()
    new_hd = control_name(new_hd, keep_list)
    new_hd['INSTNM'] = new_hd['INSTNM'].str.upper()
    
    #new_hd = hdegofr1_name(new_hd)
    #new_hd = instsize_name(new_hd)

In [ ]:
    # clean values and replace zeros with np.nan.  save the output as a .pkl for future calling
    new_hd = new_hd.replace(0,np.nan)
    new_hd.insert(1, 'YEAR', year)
    new_hd.set_index('UNITID')
    pklpath = r'/home/u1033387/FinalProject/pickles/'
    new_hd.to_pickle(pklpath+'hd{}.pkl'.format(year))
    

In [ ]:
# combine different costs including tuition, fees, and books and services into respective columns pertaining to in-state, out-of-state, full-time, and part-time students.
def colcompile (file):
    new_ic = pd.DataFrame()
    new_ic['UNITID'] = file['UNITID']
    new_ic['FTIS'] = file['ISUGT']+ file['ISUGF']+ file['BS']
    new_ic['FTOS'] = file['OSUGT']+file['OSUGF']+file['BS']
    new_ic['PTIS (9CH)'] = (file['ISUGPHC']*9)+file['ISUGF']+file['BS']
    new_ic['PTOS (9CH)'] = (file['OSUGPHC']*9)+file['OSUGF']+file['BS']
    return new_ic

In [ ]:
# some files came with periods in all the blanks, so they are replaced with zeros here and converted to int
def nopertoint (file):
    file = file.replace('.', 0)
    for column in file.columns.values:
        file[column] = file[column].astype(int, errors='ignore')
    return file

In [ ]:
# the ic tables contain information about costs of attendance for each school.  Columns are renamed to a readable format and unneccasry columns are done away with.
def iccolumns(file, year, keep_list):
    iccoldict = {'TUITION2':'ISUGT','FEE2':'ISUGF','HRCHG2':'ISUGPHC','TUITION3':'OSUGT','FEE3':'OSUGF','HRCHG3':'OSUGPHC','CHG4AY2':'BS'}
    for column in iccoldict:
        for x in range(len(iccoldict)+(len(keep_list)-len(iccoldict))):
            if column == file.columns.values[x]:
                file.columns.values[x] = iccoldict[column]
    return file

In [ ]:
# for each year the ic files are cleaned, the above functions are used, and the output is pickled.
def iccleaner(year):
    import pandas as pd
    ic = pd.read_csv('/home/u1033387/FinalProject/histdata/ic{}_ay.csv'.format(year),encoding='utf-8', index_col=False)
    for x in range(len(ic.columns.values)):
        ic.columns.values[x] = ic.columns.values[x].upper()
    keep_list = ['UNITID','TUITION2','FEE2','HRCHG2','TUITION3','FEE3','HRCHG3','CHG4AY2']
    ic = ic[keep_list].copy()
    
    ic = iccolumns(ic, year, keep_list)
    ic = nopertoint(ic)
    new_ic = colcompile(ic)
    new_ic = new_ic.replace(0, np.nan)
    new_ic.insert(1, 'YEAR', int(year))
    new_ic.set_index('UNITID')
#     return new_ic
    pklpath = r'/home/u1033387/FinalProject/pickles/'
    new_ic.to_pickle(pklpath+'ic{}_ay.pkl'.format(year))
    
    # this function takes the year format from yyyy1 to yy1yy2 format (ie 2020 becomes 1920, 2015 becomes 1516) 
def yearconv (year):
    iyear = int(year)
    year1 = str(iyear-1)
    year1 = year1[-2:]
    year2 = str(iyear)
    year2 = year2[-2:]
    iyear = year1+year2
    return iyear

In [ ]:
# create new column in sfa table (table with grant and loan info) that has the average loan and grant amounts by dividing total contributions by sudents who used contributions
def sfamath (file):
    file = file.replace(0, np.nan)
    file.insert(4, 'UPGRNTA', file['UPGRNTT']/file['UPGRNTN'])
    file.insert(7, 'UFLOANA', file['UFLOANT']/file['UFLOANN'])
    return file

In [ ]:
# the sfa file for 2008 used different column names so they were renamed here
def conv2008 (file):
    keep_list = ['UNITID','SCUGRAD', 'PGRNT_N', 'PGRNT_A', 'FLOAN_N', 'FLOAN_A']
    sfa_dict_08 = {'PGRNT_N':'UPGRNTN', 'PGRNT_A':'UPGRNTA', 'FLOAN_N':'UFLOANN', 'FLOAN_A':'UFLOANA'}
    file = file[keep_list].copy()
    for column in sfa_dict_08:
        for x in range(len(sfa_dict_08)+2):
            if column == file.columns.values[x]:
                file.columns.values[x] = sfa_dict_08[column]
    file = file.replace(0, np.nan)
    file.insert(3, 'UPGRNTT', file['UPGRNTA']*file['UPGRNTN'])
    file.insert(6, 'UFLOANT', file['UFLOANA']*file['UFLOANN'])
    return file
    

In [ ]:
# 2007 was also an inconsistent format
def conv2007 (file):
    keep_list = ['UNITID','SCFY2','SCFA2','FGRNT_N','FGRNT_A','LOAN_N','LOAN_A']
    sfa_dict_07 = {'FGRNT_N':'UPGRNTN','FGRNT_A':'UPGRNTA','LOAN_N':'UFLOANN','LOAN_A':'UFLOANA'}
    file = file[keep_list].copy()
    for column in sfa_dict_07:
        for x in range(len(sfa_dict_07)+(len(keep_list)-len(sfa_dict_07))):
            if column == file.columns.values[x]:
                file.columns.values[x] = sfa_dict_07[column]
    file = file.replace(0, np.nan)
    file.insert(3, 'UPGRNTT', file['UPGRNTA']*file['UPGRNTN'])
    file.insert(6, 'UFLOANT', file['UFLOANA']*file['UFLOANN'])
    file.insert(1, 'SCUGRAD', file['SCFA2'].fillna(file['SCFA2']))
    file.drop(['SCFY2', 'SCFA2'], axis=1, inplace=True)
    return file